# Getting the Data

## Get the Necessary Libraries

In [2]:
import requests
import zipfile
import io
import os

## Define URL and Filenames

In [3]:
url = "https://webfs.oecd.org/pisa2022/STU_QQQ_SAS.zip"
zip_filename = "SAS_STU_QQQ.zip"
target_extracted_file = "CY08MSP_STU_QQQ.sas7bdat"

## Download the File (With Stream Enable)

In [6]:
try:
    response = requests.get(url, stream=True)
    response.raise_for_status() # Check for download errors

    # Download in chunks to save memory and show progress
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024 * 1024 # 1MB chunks
    wrote = 0

    with open(zip_filename, 'wb') as f:
        for data in response.iter_content(block_size):
            wrote = wrote + len(data)
            f.write(data)
            # Simple text progress bar
            if total_size > 0:
                percent = wrote / total_size * 100
                print(f"\rDownloading: {percent:.1f}% ({wrote//(1024*1024)} MB)", end="")
    
    print(f"\nDownload complete! Saved as {zip_filename}")
except Exception as e:
    print(f"\nAn error occurred: {e}")

Downloading: 100.0% (737 MB)
Download complete! Saved as SAS_STU_QQQ.zip


## Extract The File

In [10]:
print(f"Extracting {target_extracted_file}...")
with zipfile.ZipFile(zip_filename, 'r') as z:
    # Check if the file exists in the zip
    # Note: Sometimes the internal name varies slightly, so we list files first to be safe
    file_list = z.namelist()
    print(file_list)
    # Look for the .sas7bdat file
    sas_file = [f for f in file_list if f.endswith('.SAS7BDAT')][0]
    
    z.extract(sas_file, ".")
    print(f"Success! Extracted: {sas_file}")
    
    # Optional: Rename to the standard name if it differs
    if sas_file != target_extracted_file:
        os.rename(sas_file, target_extracted_file)
        print(f"Renamed to: {target_extracted_file}")


Extracting CY08MSP_STU_QQQ.sas7bdat...
['CY08MSP_STU_QQQ.FORMAT.SAS', 'CY08MSP_STU_QQQ.SAS7BDAT']
Success! Extracted: CY08MSP_STU_QQQ.SAS7BDAT
Renamed to: CY08MSP_STU_QQQ.sas7bdat


# Data Cleaning

## Get the Necessary Libraries

In [1]:
%pip install pyreadstat

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyreadstat
import pandas as pd
import numpy as np

## Filter data for Philippines

In [4]:
df, meta = pyreadstat.read_sas7bdat(target_extracted_file)

ph_df = df[df['CNT'] == 'PHL'].copy()
print(f"Filtered for Philippines. Student count: {len(ph_df)}")

Filtered for Philippines. Student count: 7193


## Fill ANXMATH
Since we don't have ANXMATH for PH, what we'll do is to try to re-create the values, by doing the following:

1. Search by "Question Text": Variable names (like ST246) change, but the question text (e.g., "I get very tense") stays the same. We will search the metadata labels.
2. Use a Proxy (Self-Efficacy): If Anxiety is truly missing, Math Self-Efficacy (MATHEFF) is scientifically proven to be the inverse of Anxiety (Low Efficacy $\approx$ High Anxiety).
3. Check for "General Anxiety": Sometimes it's labeled under general well-being (STRRESS, etc.)

In [5]:
# 1. Search the Metadata for keywords
print("Searching metadata for 'Anxiety', 'Worry', 'Tense', or 'Math'...")

# Get the map of Column Name -> Description
col_labels = meta.column_names_to_labels

found_vars = []
keywords = ['worry', 'tense', 'nervous', 'helpless', 'anxiety', 'fear', 'afraid']

for col_name, description in col_labels.items():
    # Check if any keyword is in the description (case insensitive)
    if any(k in description.lower() for k in keywords):
        # Check if the column actually exists in our PH DataFrame
        if col_name in ph_df.columns:
            # Check if it's not all Empty/NaN
            non_null_count = ph_df[col_name].notna().sum()
            if non_null_count > 0:
                found_vars.append((col_name, description, non_null_count))

# Print what we found
print(f"\nFound {len(found_vars)} potential Anxiety/Emotion variables:")
for v in found_vars:
    print(f" - {v[0]}: {v[1]} (Non-null rows: {v[2]})")

# --- Special Check for the Best Proxy: Self-Efficacy ---
# If Anxiety is missing, Low Self-Efficacy is the best substitute.
print("\nChecking for 'Self-Efficacy' (Confidence) variables:")
efficacy_cols = [c for c in ph_df.columns if 'MATHEFF' in c or 'ST188' in c]
for c in efficacy_cols:
     print(f" - {c} (Rows: {ph_df[c].notna().sum()})")

Searching metadata for 'Anxiety', 'Worry', 'Tense', or 'Math'...

Found 13 potential Anxiety/Emotion variables:
 - ST322Q07JA: How often:  I feel nervous/anxious when I don't have my [digital device] near me. (Non-null rows: 5776)
 - ST345Q01JA: Agree/disagree: I get nervous easily. (Non-null rows: 3444)
 - ST345Q03JA: Agree/disagree: I worry about many things. (Non-null rows: 3434)
 - ST345Q07JA: Agree/disagree: I feel nervous about approaching exams. (Non-null rows: 3420)
 - ST345Q10JA: Agree/disagree: I am afraid of many things. (Non-null rows: 3490)
 - ST313Q05JA: Agree/disagree: I stay calm even in tense situations. (Non-null rows: 3446)
 - ST292Q01JA: Agree/disagree: I often worry that it will be difficult for me in mathematics classes. (Non-null rows: 5390)
 - ST292Q02JA: Agree/disagree: I get very tense when I have to do mathematics homework. (Non-null rows: 5438)
 - ST292Q03JA: Agree/disagree: I get very nervous doing mathematics problems. (Non-null rows: 5357)
 - ST292Q04JA: 

## ANXMATH Feature Engineering

The variables ST292Q01JA through ST292Q05JA are the actual raw questions PISA uses to calculate the ANXMAT index. The reason ANXMAT was empty in your file is likely because the aggregation wasn't pre-calculated.

### We combine this with MATHEFF (Self-Efficacy)
1. Scientific Validity: Anxiety (Fear) and Self-Efficacy (Confidence) are two sides of the same coin. High Anxiety usually means Low Confidence.
2. Data Coverage (The "Smart Fill"): Due to "Matrix Sampling," Student A might have answered the Anxiety questions (ST292), while Student B answered the Confidence questions (MATHEFF/ST188). By using both, we can "triangulate" the risk for far more students.

In [6]:
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

# 1. Define the Raw Columns you found
anxiety_cols = ['ST292Q01JA', 'ST292Q02JA', 'ST292Q03JA', 'ST292Q04JA', 'ST292Q05JA']

# 2. Check and Fix Directions
# In PISA: 1=Strongly Agree (Bad), 4=Strongly Disagree (Good)
# We want: 4=High Anxiety (Bad), 1=Low Anxiety (Good)
print("Processing Raw Anxiety Columns...")
for col in anxiety_cols:
    # Fill tiny gaps with median first (for students who missed just 1 question)
    ph_df[col] = ph_df[col].fillna(ph_df[col].median())
    
    # Invert the scale (Formula: New_Score = 5 - Old_Score)
    ph_df[f'{col}_FLIPPED'] = 5 - ph_df[col]

# 3. Create the "Custom Anxiety Index" (Average of flipped items)
flipped_cols = [f'{c}_FLIPPED' for c in anxiety_cols]
ph_df['ANXMAT_NEW'] = ph_df[flipped_cols].mean(axis=1)

print(f"Created 'ANXMAT_NEW'. Valid values: {ph_df['ANXMAT_NEW'].notna().sum()}")

# --- COMBINING WITH MATHEFF (Smart Imputation) ---

if 'MATHEFF' in ph_df.columns:
    print("\nIntegrating MATHEFF for Smart Imputation...")
    
    # Define columns to use for the "Pattern Filling"
    impute_cols = ['ANXMAT_NEW', 'MATHEFF']
    
    # Initialize the Smart Imputer
    imputer = IterativeImputer(max_iter=10, random_state=42)
    
    # Create a mask for rows that have at least one value (don't impute totally empty rows)
    mask = ph_df[impute_cols].notna().any(axis=1)
    
    # This magically fills missing Anxiety using the pattern found in Efficacy!
    ph_df.loc[mask, impute_cols] = imputer.fit_transform(ph_df.loc[mask, impute_cols])
    
    print("Smart Imputation Complete: Used Confidence to predict missing Anxiety.")

else:
    print("\nMATHEFF column not found directly. Using simple median fill.")
    ph_df['ANXMAT_NEW'] = ph_df['ANXMAT_NEW'].fillna(ph_df['ANXMAT_NEW'].median())

# 4. Finalize
ph_df['ANXMAT'] = ph_df['ANXMAT_NEW']
print(f"Final ANXMAT ready. Rows: {len(ph_df)}")

Processing Raw Anxiety Columns...
Created 'ANXMAT_NEW'. Valid values: 7193

Integrating MATHEFF for Smart Imputation...
Smart Imputation Complete: Used Confidence to predict missing Anxiety.
Final ANXMAT ready. Rows: 7193


## Apply Mappings

In [7]:
ph_df['Gender'] = ph_df['ST004D01T'].map({1.0: 'Female', 2.0: 'Male'})

In [8]:
# Save to Pickle (Fastest, preserves all Python data types)
ph_df.to_pickle("ph_pisa_2022_filtered.pkl")

# Audit: Check for Official PISA indices

In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2. List of Official PISA 2022 Derived Indices we hope to find
# (We already fixed ANXMAT, so we use our custom one for that)
potential_features = {
    'ANXMAT': 'Math Anxiety (Custom/Fixed)', 
    'BELONG': 'Sense of Belonging', 
    'ESCS': 'Socio-Economic Status', 
    'TEACHSUP': 'Teacher Support', 
    'BULLIED': 'Exposure to Bullying',
    'GROWMIND': 'Growth Mindset',
    'ICTRES': 'ICT Resources',
    'PERCOMP': 'Perceived Competence (Alternative to Mindset?)'
}

print("--- AUDIT: Checking for Official PISA Indices ---")
valid_features = []

for col, desc in potential_features.items():
    if col in ph_df.columns:
        # Check if it has data (not just empty column)
        count = ph_df[col].notna().sum()
        if count > 0:
            print(f"✅ FOUND: {col} ({desc}) - {count} rows")
            valid_features.append(col)
        else:
            print(f"❌ EMPTY: {col} exists but is null.")
    else:
        print(f"❌ MISSING: {col} ({desc})")

# 3. Update the features list dynamically
print(f"\nFinal Feature Set for Model: {valid_features}")

--- AUDIT: Checking for Official PISA Indices ---
✅ FOUND: ANXMAT (Math Anxiety (Custom/Fixed)) - 7193 rows
✅ FOUND: BELONG (Sense of Belonging) - 6973 rows
✅ FOUND: ESCS (Socio-Economic Status) - 7167 rows
✅ FOUND: TEACHSUP (Teacher Support) - 6755 rows
✅ FOUND: BULLIED (Exposure to Bullying) - 6990 rows
❌ MISSING: GROWMIND (Growth Mindset)
✅ FOUND: ICTRES (ICT Resources) - 7152 rows
❌ MISSING: PERCOMP (Perceived Competence (Alternative to Mindset?))

Final Feature Set for Model: ['ANXMAT', 'BELONG', 'ESCS', 'TEACHSUP', 'BULLIED', 'ICTRES']
